In [1]:
from datetime import datetime
import functools
from IPython.display import HTML
import jax
from jax import numpy as jp
import numpy as np
from typing import Any, Dict, Sequence, Tuple, Union
from brax import base
from brax import envs
from brax import math
from brax.base import Base, Motion, Transform
from brax.envs.base import Env, PipelineEnv, State
from brax.mjx.base import State as MjxState
from brax.training.agents.apg import train as apg
from brax.training.agents.apg import networks as apg_networks
from brax.io import html, mjcf, model
from etils import epath
from flax import struct
from matplotlib import pyplot as plt
import mediapy as media
from ml_collections import config_dict
import mujoco
from mujoco import mjx
from jax import vmap
import jax.random
from jax import lax

In [2]:
from utils.SimpleConverter import SimpleConverter
from agent_mimic_env.agent_template import HumanoidTemplate
from agent_mimic_env.agent_eval_template import HumanoidEvalTemplate
from agent_mimic_env.agent_test_apg import HumanoidAPGTest
from utils.util_data import *
from copy import deepcopy

In [3]:
import agent_mimic_env
from agent_mimic_env import register_mimic_env


In [4]:

import distutils.util
import os
xla_flags = os.environ.get('XLA_FLAGS', '')
xla_flags += ' --xla_gpu_triton_gemm_any=True'
os.environ['XLA_FLAGS'] = xla_flags
print('Setting environment variable to use GPU rendering:')
%env MUJOCO_GL=egl

Setting environment variable to use GPU rendering:
env: MUJOCO_GL=egl


In [5]:
import os
os.environ["XLA_PYTHON_CLIENT_MEM_FRACTION"] = "0.8" # 0.9 causes too much lag. 
from datetime import datetime
import functools

# Math
import jax.numpy as jp
import numpy as np
import jax
from jax import config # Analytical gradients work much better with double precision.
config.update("jax_debug_nans", True)
config.update("jax_enable_x64", True)
config.update('jax_default_matmul_precision', jax.lax.Precision.HIGH)
from brax import math

# Sim
import mujoco
import mujoco.mjx as mjx

# Brax
from brax import envs
from brax.base import Motion, Transform
from brax.io import mjcf
from brax.envs.base import PipelineEnv, State
from brax.mjx.pipeline import _reformat_contact
from brax.training.acme import running_statistics
from brax.io import model

# Algorithms
# from brax.training.agents.apg import train as apg
# from brax.training.agents.apg import networks as apg_networks
from brax.training.agents.ppo import train as ppo

# Supporting
from etils import epath
import mediapy as media
import matplotlib.pyplot as plt
from ml_collections import config_dict
from typing import Any, Dict

In [6]:
from jax.lib import xla_bridge
print(xla_bridge.get_backend().platform)

gpu


In [7]:
import yaml
from box import Box
# Path to your YAML file
yaml_file_path = 'config_params/punch.yaml'
# Load the YAML file
with open(yaml_file_path, 'r') as file:
    args = Box(yaml.safe_load(file))

In [8]:
print(args)

{'num_envs': 256, 'num_eval_envs': 32, 'lr': 0.0003, 'max_it': 1000, 'max_grad_norm': 0.4, 'seed': 0, 'system_config': 'humanoid', 'demo_replay_mode': 'threshold', 'threshold': 0.4, 'normalize_observations': True, 'cycle_len': 130, 'ep_len': 65, 'ep_len_eval': 54, 'use_lr_scheduler': True, 'reward_scaling': 0.02, 'rot_weight': 0.5, 'vel_weight': 0.01, 'ang_weight': 0.01, 'deep_mimic_reward_weights': {'w_p': 0.3, 'w_v': 0.1, 'w_e': 0.5, 'w_c': 0.1}, 'deep_mimic_weights_factors': {'w_pose': 2, 'w_angular': 0.005, 'w_efector': 5, 'w_com': 100}, 'ref': 'motions/humanoid3d_punch_duplicated.txt'}


In [9]:
args.deep_mimic_reward_weights.w_p

0.3

In [10]:
from agent_mimic_env.pds_controllers_agents import feedback_pd_controller, stable_pd_controller_action


In [11]:
env_replay,env_eval,env,env_apg=register_mimic_env(args)


#for the env
jit_reset = jax.jit(env.reset)
#jit_step = jax.jit(env.step)
jit_step_custom_env = jax.jit(env.step_custom)
env.set_pd_callback(stable_pd_controller_action)


# #for the eval
jit_reset_eval = jax.jit(env_eval.reset)
jit_step_custom_eval = jax.jit(env_eval.step_custom)
env_eval.set_pd_callback(stable_pd_controller_action)

#for the replay
jit_reset_replay = jax.jit(env_replay.reset)
jit_step_replay= jax.jit(env_replay.step)

#for apg
# jit_reset_apg = jax.jit(env_apg.reset)
# jit_step_custom_apg= jax.jit(env_apg.step_custom)
# env_apg.set_pd_callback(stable_pd_controller_action)





qpos init (130, 35)
qvel init (130, 34)


In [12]:
env_eval.sys.opt

Option(timestep=0.002, impratio=Array(100., dtype=float64, weak_type=True), tolerance=Array(1.e-08, dtype=float64, weak_type=True), ls_tolerance=Array(0.01, dtype=float64, weak_type=True), gravity=Array([ 0.  ,  0.  , -9.81], dtype=float64), wind=Array([0., 0., 0.], dtype=float64), density=Array(0., dtype=float64, weak_type=True), viscosity=Array(0., dtype=float64, weak_type=True), has_fluid_params=False, integrator=<IntegratorType.EULER: 0>, cone=<ConeType.PYRAMIDAL: 0>, jacobian=<JacobianType.AUTO: 2>, solver=<SolverType.NEWTON: 2>, iterations=1, ls_iterations=4, disableflags=<DisableBit.EULERDAMP: 16384>)

In [13]:
print(env_eval.sys.opt.timestep)
print(env_eval.dt)

0.002
0.02


In [14]:
print(env_eval.rollout_lenght)
print(env_eval.cycle_len)
print(env_eval.rot_weight)
print(env_eval.vel_weight)
print(env_eval.ang_weight)
print(env_eval.reward_scaling)
print(env_eval.dt)
# print(env_eval.sys.dt)

print(env.rollout_lenght)
print(env.cycle_len)
print(env.rot_weight)
print(env.vel_weight)
print(env.ang_weight)
print(env.reward_scaling)
print(env.dt)
print(env.dt)
print(env.err_threshold)

65
130
0.5
0.01
0.01
0.02
0.02
65
130
0.5
0.01
0.01
0.02
0.02
0.02
0.4


In [15]:
# initialize the state

#for the replay
# state = jit_reset_replay(jax.random.PRNGKey(0))
# rollout = [state.pipeline_state]

#for the eval
state = jit_reset_eval(jax.random.PRNGKey(0))
rollout = [state.pipeline_state]
#for env
# state = jit_reset(jax.random.PRNGKey(0))
# rollout = [state.pipeline_state]

#for apg test
# state = jit_reset_apg(jax.random.PRNGKey(0))
# rollout = [state.pipeline_state]

# grab a 500 steps
n_steps = 500

In [16]:
print(state.metrics)
print(state.info)

{'fall': Array(0., dtype=float64), 'pose_error': Array(0., dtype=float64), 'reference_angular': Array(0., dtype=float64, weak_type=True), 'reference_position': Array(0., dtype=float64, weak_type=True), 'reference_rotation': Array(0., dtype=float64, weak_type=True), 'reference_velocity': Array(0., dtype=float64, weak_type=True), 'step_index': Array(0., dtype=float64)}
{'reward_tuple': {'reference_angular': Array(0., dtype=float64, weak_type=True), 'reference_position': Array(0., dtype=float64, weak_type=True), 'reference_rotation': Array(0., dtype=float64, weak_type=True), 'reference_velocity': Array(0., dtype=float64, weak_type=True)}, 'rng': Array([0, 0], dtype=uint32), 'steps': Array(0., dtype=float64, weak_type=True)}


In [17]:
id = 9

id =(195 % 195) /195
id
# cy = (66%65)
# cy

0.0

In [18]:
for i in range(env_eval.cycle_len-1):
    
    #print(i)
    ctrl = -0.1 * jp.ones(env_eval.sys.nu)
    #ctrl = -0.1 * jp.ones(env.sys.nu)
    #ctrl = -0.1 * jp.ones(env_replay.sys.nu)
    #ctrl = -0.1 * jp.ones(env_apg.sys.nu)
    #time
    #time = state.pipeline_state.time
    
    #print('time: ',state.pipeline_state.time)    
         
    #state = jit_step_replay(state, ctrl)
    state = jit_step_custom_eval(state, ctrl)
    #state = jit_step(state, ctrl)
    #state = jit_step_custom_env(state,ctrl)
    #state = jit_step_custom_apg(state,ctrl)
    
    #print(state.pipeline_state.geom_xpos)
    #if state.done:
     #   print(time)
      #  break
    #print("Is done", state.done)
    # print("Is rewards", state.reward)
    #print('is step', state.metrics['step_index'])
    #print("reward tuple", state.info['reward_tuple'])
    # print("Is ", state.metrics['pose_error'])
    
    rollout.append(state.pipeline_state)

In [19]:
from some_math.quaternion_diff import *

In [20]:
#6 right writs 9 left writs 12 right ankle, 15 left ankle

print(state.pipeline_state.geom_xpos)


#index 16

[[ 0.          0.          0.        ]
 [-0.76396419 -0.78976515  0.21619712]
 [-1.04000024 -0.78195297  0.15583256]
 [-1.30681118 -0.75183099  0.10236905]
 [-1.13935921 -0.48344391  0.18529264]
 [-1.06622057 -0.277143    0.14845597]
 [-1.02709451 -0.17955042  0.05762036]
 [-1.06148542 -1.05576846  0.20062306]
 [-0.91707704 -1.05498415  0.20858777]
 [-0.84466332 -0.96340444  0.13325067]
 [-0.49973614 -0.79285372  0.16490164]
 [-0.20702161 -0.93694818  0.26864131]
 [-0.05323907 -1.0206897   0.41655846]
 [-0.5259336  -0.94942759  0.13867762]
 [-0.18135635 -1.03268904  0.15984826]
 [ 0.03819731 -1.06044028  0.258987  ]]


In [21]:
#HTML(html.render(env_replay.sys.tree_replace({'opt.timestep': env_replay.dt}), rollout))
HTML(html.render(env_eval.sys.tree_replace({'opt.timestep': env_eval.dt}), rollout))
#HTML(html.render(env.sys.tree_replace({'opt.timestep': env.dt}), rollout))
#HTML(html.render(env_apg.sys.tree_replace({'opt.timestep': env_apg.dt}), rollout))
